# **CA4. Multiprocessing**
**Sulu Chi Yahir Benjamin\
Data Engineering\
Universidad Politécnica de Yucatán\
Ucú, Yucatán, México\
2109145@upy.edu.mx**


# Part 4. Matrix and Vector Computations

13. Read about Broadcasting with Arrays on the chapter Computation on Arrays:\
Broadcasting from Python Data Science Handbook (J. VandePlas, 2016):\
**Link:** https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computationon-arrays-broadcasting.html.

The description outlines NumPy's pivotal components for streamlined computation, emphasizing its reliance on vectorized operations and the groundbreaking concept of broadcasting. It delves into the mechanics of universal functions (ufuncs), which serve as the engine behind rapid array computations. By leveraging unary and binary operations like arithmetic, trigonometry, and logarithms, ufuncs circumvent Python's inherent loop sluggishness, catapulting efficiency.

Furthermore, the discourse accentuates the specialized functionalities of ufuncs, including output specification, aggregates, and outer products, which bolster NumPy's computational prowess. Broadcasting emerges as a standout attribute, enabling seamless vectorized operations across arrays of disparate shapes sans explicit data replication. This streamlined approach not only simplifies code but also augments performance, adhering to NumPy's ethos of computational efficiency.

The narrative delineates the rules guiding broadcasting in NumPy, such as dimension extension and avoidance of data duplication, crucial for optimizing computational and memory resources in high-performance computing (HPC) environments. These rules underpin the versatility of NumPy's binary universal functions, catering to diverse mathematical operations with finesse.

Moreover, illustrative examples underscore broadcasting's utility in data processing, scientific analysis, and machine learning tasks, showcasing its role in operations like array centering and grid-based function computation. In the realm of HPC and matrix/vector computations, broadcasting emerges as an indispensable tool for executing vectorized operations efficiently. Its role in simplifying coding, enhancing computational speed, and fostering flexibility in algorithmic design underscores its significance in numerical computing landscapes.

14. Read Rewriting the particle simulator in Numpy on Chapter 2: Fast Array Operations
with Numpy and Pandas (pp. 68) from the book G. Lenaro (2017). Python high
Performance. Second Edition. UK: Packt Publishing Ltd. Implement the
improvements on the particle simulator using NumPy. Show that both
implementations scale linearly with particle size, but the runtime in the pure Python
version grows much faster than the NumPy version. 

In [1]:
!pip install numpy

   ---------------------------------------- 97.0/97.0 kB 923.4 kB/s eta 0:00:00


In [ ]:
import numpy as np
from random import uniform

In [ ]:
class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel

class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles

    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)

        for _ in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.ang_vel * v_x
                d_y = timestep * p.ang_vel * v_y
                p.x += d_x
                p.y += d_y

    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])

        for _ in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * ang_vel_i[:, np.newaxis] * v_i
            r_i += d_i

        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

def benchmark(npart=100, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)

    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)

# Example of benchmarking
if __name__ == "__main__":
    from timeit import timeit

    print("Benchmarking with 100 particles:")
    print("Python version:", timeit("benchmark(100, 'python')", globals=globals(), number=1), "seconds")
    print("NumPy version:", timeit("benchmark(100, 'numpy')", globals=globals(), number=1), "seconds")

15. Explain how to optain the optimal performance with numexpr. Read the section
Reaching optimal performance with numexpr, pp. 72 from the previous reference.
Implement it and measure the execution time.

NumPy revolutionizes computational efficiency by harnessing array broadcasting and memory optimization, particularly vital for handling vast datasets. Comparative benchmarks vividly demonstrate the exponential performance gains as one transitions from conventional Python loops to NumPy's lightning-fast vectorized operations, especially evident with larger datasets.

Expanding upon NumPy's groundwork, Numexpr elevates these advancements by fine-tuning array expressions, curbing memory usage, and tapping into multiple CPU cores for parallel processing. By sidestepping intermediate array storage and maximizing CPU cache utilization, Numexpr achieves remarkable speed enhancements in operations involving extensive arrays.

These strides shine in practical scenarios like norm calculations and particle displacement analyses, where Numexpr's knack for compiling and streamlining complex expressions in real-time leads to substantial performance leaps beyond traditional NumPy methods.

In [ ]:
!pip install numexpr

In [ ]:
import numpy as np
from random import uniform
import numexpr as ne

In [ ]:
class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel
class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles
    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        for i in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.ang_vel * v_x
                d_y = timestep * p.ang_vel * v_y
                p.x += d_x
                p.y += d_y
    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])
        for i in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * ang_vel_i[:, np.newaxis] * v_i
            r_i += d_i
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]
 
    def evolve_numexpr(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])
 
        for i in range(nsteps):
            norm_i = ne.evaluate("sqrt((r_i ** 2).sum(axis=1))")
            v_i = r_i[:, [1, 0]]
            v_i[:, 0] = ne.evaluate("-v_i[:, 0]")
            v_i = ne.evaluate("v_i / norm_i[:, None]")
            d_i = ne.evaluate("timestep * ang_vel_i[:, None] * v_i")
            r_i = ne.evaluate("r_i + d_i")
 
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]
 
def benchmark(npart=100000, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)
    elif method == 'numexpr':
        simulator.evolve_numpy(0.1)
 
# Example of benchmarking
if __name__ == "__main__":
    from timeit import timeit
    print("Benchmarking with 100000 particles:")
    print("Python version:", timeit("benchmark(100000, 'python')", globals=globals(), number=1), "seconds")
    print("NumPy version:", timeit("benchmark(100000, 'numpy')", globals=globals(), number=1), "seconds")
    print("NumExpr version:", timeit("benchmark(100000, 'numexpr')", globals=globals(), number=1), "seconds")